In [1]:
"""Viterbi Algorithm for inferring the most likely sequence of states from an HMM.

Patrick Wang, 2021
"""

'Viterbi Algorithm for inferring the most likely sequence of states from an HMM.\n\nPatrick Wang, 2021\n'

In [2]:
from typing import Sequence, Tuple, TypeVar
import numpy as np

In [16]:
# import nltk

# nltk.download("brown")
# nltk.download("tagsets")
# nltk.download("universal_tagset")
tagged_corpus = nltk.corpus.brown.tagged_sents(tagset="universal")[:10000]
infered_corpus = nltk.corpus.brown.tagged_sents(tagset="universal")[10150:10153]

In [30]:
print(tagged_corpus[:])

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

In [29]:
for tagged_sent in tagged_corpus[:]:
    # print(tagged_sent)
    print(tagged_sent[0][0], tagged_sent[0][1])

The DET
The DET
The DET
`` .
The DET
It PRON
The DET
Merger NOUN
However ADV
The DET
It PRON
Implementation NOUN
It PRON
The DET
`` .
The DET
Nevertheless ADV
`` .
The DET
Wards NOUN
The DET
`` .
Regarding ADP
The DET
Ask VERB
On ADP
Four NUM
( .
Fulton NOUN
The DET
Mayor NOUN
His DET
The DET
They PRON
Attorneys NOUN
The DET
It PRON
The DET
The DET
Henry NOUN
Hartsfield NOUN
His DET
The DET
He PRON
Georgia NOUN
Robert NOUN
State NOUN
In ADP
Despite ADP
When ADV
The DET
Five NUM
The DET
Sam NOUN
Caldwell's NOUN
He PRON
When ADV
The DET
Before ADP
Gov. NOUN
Vandiver NOUN
Construction NOUN
Meanwhile ADV
The DET
A DET
A DET
The DET
Vandiver NOUN
The DET
Schley NOUN
Pelham NOUN
While ADP
A DET
As ADP
Pelham NOUN
A DET
Rep. NOUN
Barber NOUN
But CONJ
The DET
It PRON
Colquitt NOUN
-- .
The DET
Davis NOUN
Ordinary NOUN
`` .
`` .
I PRON
The DET
It PRON
The DET
During ADP
Ordinary NOUN
Many ADJ
Sheriff NOUN
`` .
`` .
Austin NOUN
-- .
Daniel NOUN
Under ADP
But CONJ
Daniel NOUN
He PRON
It PRON
The 

In [5]:
# Call my below viterbi function with the following Args:
#     obs: An iterable of ints representing observations.
#     pi: A 1D numpy array of floats representing initial state probabilities.
#     A: A 2D numpy array of floats representing state transition probabilities.
#     B: A 2D numpy array of floats representing emission probabilities.
obs = [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
pi = np.array([0.5, 0.5])
A = np.array([[0.7, 0.3], [0.3, 0.7]])
B = np.array([[0.9, 0.1], [0.2, 0.8]])
viterbi(obs, pi, A, B)

([0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 4.080261582028813e-06)

In [3]:
Q = TypeVar("Q")
V = TypeVar("V")


def viterbi(
    obs: Sequence[int],
    pi: np.ndarray[Tuple[V], np.dtype[np.float_]],
    A: np.ndarray[Tuple[Q, Q], np.dtype[np.float_]],
    B: np.ndarray[Tuple[Q, V], np.dtype[np.float_]],
) -> tuple[list[int], float]:
    """Infer most likely state sequence using the Viterbi algorithm.

    Args:
        obs: An iterable of ints representing observations.
        pi: A 1D numpy array of floats representing initial state probabilities.
        A: A 2D numpy array of floats representing state transition probabilities.
        B: A 2D numpy array of floats representing emission probabilities.

    Returns:
        A tuple of:
        * A 1D numpy array of ints representing the most likely state sequence.
        * A float representing the probability of the most likely state sequence.
    """
    N = len(obs)
    Q, V = B.shape  # num_states, num_observations

    # d_{ti} = max prob of being in state i at step t
    #   AKA viterbi
    # \psi_{ti} = most likely state preceeding state i at step t
    #   AKA backpointer

    # initialization
    log_d = [np.log(pi) + np.log(B[:, obs[0]])]
    log_psi = [np.zeros((Q,))]

    # recursion
    for z in obs[1:]:
        log_da = np.expand_dims(log_d[-1], axis=1) + np.log(A)
        log_d.append(np.max(log_da, axis=0) + np.log(B[:, z]))
        log_psi.append(np.argmax(log_da, axis=0))

    # termination
    log_ps = np.max(log_d[-1])
    qs = [-1] * N
    qs[-1] = int(np.argmax(log_d[-1]))
    for i in range(N - 2, -1, -1):
        qs[i] = log_psi[i + 1][qs[i + 1]]

    return qs, np.exp(log_ps)